# Demo of cryptography on elliptic curves

Let us initialize an elliptic curve with the following equation

$$y^2=x^3+x+2$$

When $x$ and $y$ belong to the ring $\mathbb{Z}/5=\{0,1,2,3,4\}$ it is easy to find all solutions to the congruence $y^2\equiv x^3+x+2(\textrm{mod }5)$.

We perform both addition and multiplication in $\mathbb{Z}/5$ in the usual $\textrm{mod }5$ way. These two operations constitute a structure of a <b>field</b> on $\mathbb{Z}/5$. We denote this structure as $\mathbb{F}_{5}$. 

For example: $2+3=0$ in $\mathbb{F}_{5}$. We can write both the addition and multiplication table:

In [1]:
F5=GF(5)

In [2]:
#addition table
add=[[0]+[F5(i) for i in range(0,5)]]+[[F5(i)]+[F5(i)+F5(j) for j in range(0,5)] for i in range(0,5)];
pretty_print(matrix(add))

[0 0 1 2 3 4]
[0 0 1 2 3 4]
[1 1 2 3 4 0]
[2 2 3 4 0 1]
[3 3 4 0 1 2]
[4 4 0 1 2 3]

In [4]:
#multiplication table
mul=[[0]+[F5(i) for i in range(1,5)]]+[[F5(i)]+[F5(i)*F5(j) for j in range(1,5)] for i in range(1,5)];
pretty_print(matrix(mul))

[0 1 2 3 4]
[1 1 2 3 4]
[2 2 4 1 3]
[3 3 1 4 2]
[4 4 3 2 1]

In [58]:
E=EllipticCurve([GF(5)(1),2]);

We generate all points on $E$ over the finite field $\mathbb{F}_5$. Apart from the points of the form $(x,y)$ we have one additional point 'at infinity' which is the neutral point of the group law on $E$.

In [64]:
E.rational_points() #(x:y:1) - projective coordinates of the point (x,y); 
                    #(0:1:0) - point at infinity

[(0 : 1 : 0), (1 : 2 : 1), (1 : 3 : 1), (4 : 0 : 1)]

The group under consideration is formed from the set $E(\mathbb{F}_{5})$ which consists of $3$ points
$(1,2), (1,3)$ i $(4,0)$

and one more point at 'infinity'.
$(0:1:0)$ (the notation is different because this point has ''zero denominator'', which makes the form $(x,y)$ impossible to write.

In [11]:
P=E([1,2]) #we pick one of these points

We add the point $P$ several times to itself. It will turn out that the points listed above are multiples of $P$.
That means that the group $E(\mathbb{F}_{5})$ is cyclic and generated by $P$!

In [14]:
print(P)
print(2*P)
print(3*P)
print(4*P)

(1 : 2 : 1)
(4 : 0 : 1)
(1 : 3 : 1)
(0 : 1 : 0)


Algebraic group law on $E$ is expressed by a complicated formula which we reveal below.

In [25]:
Fx.<x>=FunctionField(GF(5))
RY.<Y>=Fx[]
Fy.<y>=Fx.extension(x^3+x+3-Y^2)
Fw.<w>=FunctionField(Fy)
RZ.<Z>=Fw[]
Fz.<z>=Fw.extension(w^3+w+3-Z^2)

In [37]:
EF=EllipticCurve([Fz(1),3])
P1=EF([x,y])
P2=EF([w,z])

(P1+P2)[0]==(1 + w + x + w^2*x + w*x^2 + 3*y*z)/(w^2 + 3*w*x + x^2)
(P1+P2)[1]==(2*y + 3*w*y + w^3*y + x*y + 3*w^2*x*y + 3*z + 4*w*z + 2*x*z + 2*w*x^2*z + 4*x^3*z)/(w^3 + 2*w^2*x + 3*w*x^2 + 4*x^3)

True

In [38]:
pretty_print(P1+P2)

((3*y/(w^2 + 3*x*w + x^2))*z + (x*w^2 + (x^2 + 1)*w + x + 1)/(w^2 + 3*x*w + x^2) : (((2*x^2 + 4)*w + 4*x^3 + 2*x + 3)/(w^3 + 2*x*w^2 + 3*x^2*w + 4*x^3))*z + (y*w^3 + 3*x*y*w^2 + 3*y*w + (x + 2)*y)/(w^3 + 2*x*w^2 + 3*x^2*w + 4*x^3) : 1)

Addition of point on $E$ is complicated algebraically but intuitive geometrically. At the same time these computations can be performed fast on the computer.

In [39]:
pretty_print(2*P1)

((4*x^4 + 2*x^2 + 4*x + 4)/(x^3 + x + 3) : ((2*x^6 + x + 4)/(x^6 + 2*x^4 + x^3 + x^2 + x + 4))*y : 1)

* In principle, a curv above could be used for cryptographic purposes but the group under consideration has only $4$ elements which is obviously not safe enough for any reasonable coding.

* What we need to change is first of all the field of definition of our curve.

* In practice, we need points of orders going into millions.

## Encoding a plain text on the curve (Koblitz method).

We encode our message $m$ as a point on the curve. The first coordinate is going to be $\kappa\cdot m+j$, where our message should be $m<p/\kappa$ and $1\leq j\leq \kappa$. This guarantees we will find a suitable point in the radnomized procedure (with chance of success $1-1/2^\kappa$. Coordinate $y$ is chosen in such a way that $y^2=x^3+ax+b$, where $a,b,$ are fixed, and $x=\kappa\cdot m+j$ for a fixed $j$.


In [40]:
#letter encoding (not reasonable, why?)
def EncodePoint(E,char,kappa=100):
    p=E.base_field().characteristic() #field characteristic
    m=ord(char) #ASCII code
    assert m <= p*1.0/kappa
    pol=E.defining_polynomial()
    for i in range(0,kappa):
        val=-pol([kappa*m+i,0,1])
        if val.is_square():
            sq=sqrt(val)
            P=E([kappa*m+i,sq,1])
            break;
        
    return P

In [41]:
EE=EllipticCurve([GF(160174168162444816244821958911111)(1),3])
EncodePoint(EE,"m")

(10901 : 42332909399061611549525231919913 : 1)

In [42]:
RealField(20)(log(EE.order(),10)) #EE.order() ~= 10^32, is a large number!

32.205

In [170]:
#unfortunately the cyclic subgroups here are of rather small orde which means we
#should choose a better curve
EE.order().factor()

2^3 * 5 * 41 * 59 * 269 * 1232450832397 * 4993151403869

Decoding procedure is very simple. We compute the floor of the expression $x/\kappa$, where $x$ is the first coordinate of a point.

In [43]:
def DecodePoint(P,kappa=100):
    x=int(P[0])
    return chr(int(x/kappa))

Exemplary coding and decoding of a point (points on the curve are not displayed)

In [44]:
alphabet=[ "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", 
"p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z" ];
EE=EllipticCurve([GF(160174168162444816244821958911111)(1),3]);
print([DecodePoint(EncodePoint(EE,x)) for x in alphabet])==alphabet

True


<b>Hint</b>: The orders of points are not always equal to the order of the group, which makes our cryptosystem weaker. It is reasonable to choose our group in such a way that there are just a few prime factors in the group order.


In [45]:
pp=[(EncodePoint(EE,x)) for x in alphabet]
[x.order()==EE.order() for x in pp]

[False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True]

# Elliptic ElGamal cryptosystem

Public key: selected elliptic curve $E:y^2=x^3+ax+b$, fixed point $P$ and point $B$ which is an $s$-multiple of $P$.
        
Private key: integer $s$.

Alice sends a message $M$ to Bob and encodes it as a sequence of symbols. For simplicity we assume that $\tilde{m}$ represents a single letter (why this is bad, explain). Symbol $\tilde{m}$ is encoded as a point on the elliptic curve $E$.

In [46]:
def EncodeElGamal(char,public_key):
    E,P,B=public_key
    m=EncodePoint(E,char)
    k=randint(1,10^10)
    M1=k*P
    M2=m+k*B
    return (M1,M2)

Bob generates his private key s and a public key for Alice.

In [48]:
s=2020; #this is secret (better to choose randomly)
E=EllipticCurve([GF(160174168162444816244821958911111)(1),3])
P=E.gens()[0] #this is one of the large order generators
BobPublicKey=(E,P,s*P) #generating a public key is fast

In [49]:
%timeit BobPublicKey=(E,P,s*P)

1000 loops, best of 3: 696 µs per loop


Now, we only need to write a decryption procedure of the ElGamal protocol.

In [50]:
def DecodeElGamal(encoded_message,secret):
    M1,M2=encoded_message
    m=M2-secret*M1
    return DecodePoint(m)

## Testing:

In [51]:
encrypted_char=EncodeElGamal("a",BobPublicKey)
print("Cryptogram:")
print(encrypted_char)

Cryptogram:
((133383931880786219793674923743411 : 134099150139626582176451473826987 : 1), (2564428787381749704048423157449 : 55030316571076707898368168964870 : 1))


In [53]:
decoded_char=DecodeElGamal(encrypted_char,s);
print("Plain text")
print(decoded_char)

Plain text
a


## Sending a longer message

In [54]:
#secret message
message="Santa Claus is coming to town!";

Preparing a message

In [55]:
#secret computation on Alice's computer
cipher=[];
for i in [0..len(message)-1]:
    cipher.append(EncodeElGamal(message[i],BobPublicKey))

Our ciphertext looks complicated enough

In [56]:
print(cipher)

[((14132705556334431680761941110726 : 77320659951762150521827460340460 : 1), (2564831799005801545927062120964 : 145470655782571136804270900956141 : 1)), ((9331444413030074282522930723029 : 28515237105276136198925521231126 : 1), (94270808769653222649546349443185 : 6968998331059861763163204283383 : 1)), ((1510366906613463424932101316179 : 74383950846419854464903191012436 : 1), (33207632288535007640416446032396 : 157011724040381235215889473838645 : 1)), ((116529027617008692725946813431971 : 108506615088573879108122566534114 : 1), (47962969080825261517542671430616 : 62921089135360218387992378056085 : 1)), ((79648026742559481536679751654853 : 80583208739896649098849246368221 : 1), (36424428948635394700528120513331 : 155471446352486592796336204538064 : 1)), ((157889292088311381370466774392856 : 115307156526318243389962397409460 : 1), (86942706919283680462689201405939 : 65749198933562169264967815972029 : 1)), ((95133270475775545636252316463531 : 44400977803625420808890761942009 : 1), (9610179

We send our cipher through a public channel. If no one guessed our secret key $s$, it will be very hard to get your hands on the message (DLP type problem).

Deciphering happens on Bob's computer with the help of the secret key $s$.

In [57]:
#Bob's deciphering procedure
plain_text=[DecodeElGamal(char,s) for char in cipher]
print(''.join(plain_text))

Santa Claus is coming to town!


Exercise: Why the encoding scheme: one symbol --> one point on the elliptic curve is not safe. How many unique points will appear in the cryptogram? Can you perform a statistical analysis on the ciphertext?

Improve the scheme by changing how the letters are converted to points on the curves.